# Ensemble Methods
## Experimenting on MNIST dataset

Let's use sklearn's implementations of the Bagging, Random Forest, Weighted Voting and Stacking models and perform experiments on MNIST dataset. First, we will use only 2 digits from the dataset and then will use all 10 digits. 

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-whitegrid") # Plot style

from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

%load_ext autoreload
%autoreload 2

In [2]:
mnist_train = pd.read_csv("data/raw/mnist_train.csv", header=None)
mnist_test = pd.read_csv("data/raw/mnist_test.csv", header=None)

C:\Users\User\AppData\Local\Temp\ipykernel_7592\3586860171.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,25

In [3]:
X_train, y_train = mnist_train.iloc[:2000, 1:], mnist_train.iloc[:2000, 0]
X_test, y_test = mnist_test.iloc[:1000, 1:], mnist_test.iloc[:1000, 0]

## 1. Bagging

In [4]:
X_train.shape

(2000, 784)

In [5]:
start = time.perf_counter()
classifier = BaggingClassifier(n_estimators=10, random_state=0)
classifier.fit(X_train, y_train)
print(f"Time taken: {time.perf_counter() - start} sec")
print("Train error rate: ", 1 - accuracy_score(classifier.predict(X_train), y_train))
print("Test error rate: ", 1 - accuracy_score(classifier.predict(X_test), y_test))

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
# ensemble_sizes = range(1, 51, 4)
ensemble_sizes = range(50, 65, 4)
train_error = []
test_error = []

for ensemble_size in ensemble_sizes:
    model = BaggingClassifier(n_estimators=ensemble_size, random_state=0)
    model.fit(X_train, y_train)
    train_error.append(1 - accuracy_score(model.predict(X_train), y_train))
    test_error.append(1 - accuracy_score(model.predict(X_test), y_test))

plt.plot(ensemble_sizes, train_error)
plt.plot(ensemble_sizes, test_error)
plt.xticks(ensemble_sizes)
plt.xlabel("Number of estimators")
plt.ylabel("Error Rate")
plt.legend(["Train", "Test"])
plt.show()

## 2. Random Forest (RF)

In [ ]:
start = time.perf_counter()
classifier = RandomForestClassifier(n_estimators=50, max_features=50, random_state=0)
classifier.fit(X_train, y_train)
print(f"Time taken: {time.perf_counter()-start} sec")
print("Train error rate: ", 1 - accuracy_score(classifier.predict(X_train), y_train))
print("Test error rate: ", 1 - accuracy_score(classifier.predict(X_test), y_test))

In [ ]:
ensemble_sizes = range(1, 51, 4)
for nr_features in [10, 50, 300, "auto"]:
    error_rates = {"train":[], "test":[]}

    for ensemble_size in ensemble_sizes:
        model = RandomForestClassifier(n_estimators = ensemble_size, random_state = 0, max_features=nr_features)
        model.fit(X_train, y_train)

        error_rates["train"].append(1 - accuracy_score(model.predict(X_train), y_train))
        error_rates["test"].append(1 - accuracy_score(model.predict(X_test), y_test))
    
    plt.plot(ensemble_sizes, error_rates["train"], label=f"{nr_features}", linestyle="--", linewidth=2)
    plt.plot(ensemble_sizes, error_rates["test"], label=f"{nr_features}", linewidth=2)

plt.xlabel("Number of trees")
plt.ylabel("Error Rate")
plt.xticks(ensemble_sizes)
plt.legend(ncol=2)
plt.show()

In [ ]:
# Returns importance of each feature
pd.DataFrame({"pixel":np.arange(28**2), "importance": model.feature_importances_}).sort_values("importance", ascending=False)

Now, let's use all the digits in the dataset. The ideal scenario:
1. find the optimal number of estimators on training dataset (e.g. using Cross-Validation),
2. train an ensemble model with optimal number of estimators (i.e. individual models) on training dataset,
3. test the trained ensemble model on testing dataset.

In [ ]:
X_train, y_train = mnist_train.iloc[:,1:], mnist_train.iloc[:,0]
X_test, y_test = mnist_test.iloc[:,1:], mnist_test.iloc[:,0]

In [ ]:
start = time.perf_counter()
classifier = BaggingClassifier(n_estimators=17, random_state=0) # Suppose, the optimal number of estimators is 17 for Bagging
classifier.fit(X_train, y_train)
print(f"Time taken: {time.perf_counter() - start} sec")
print("Train accuracy: ", accuracy_score(classifier.predict(X_train), y_train))
print("Test accuracy: ", accuracy_score(classifier.predict(X_test), y_test))

In [ ]:
start = time.perf_counter()
classifier = RandomForestClassifier(n_estimators=45, random_state=0) # Suppose, the optimal number of estimators is 45 for RF
classifier.fit(X_train, y_train)
print(f"Time taken: {time.perf_counter() - start} sec")
print("Train accuracy: ", accuracy_score(classifier.predict(X_train), y_train))
print("Test accuracy: ", accuracy_score(classifier.predict(X_test), y_test))

Here, both classifiers overfit on training data, and the test accuracy for:
1. Bagging should be around 92%, because Bagging uses all features (total: 764 features),
2. Random Forest should be around 95%, because Random Forest does not use all features.

The other 2 algorithms are implemented in similar ways as Bagging and Random Forest.

## 3. Weighted Voting

## 4. Stacking